## Imports

In [2]:
import os
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache, openai_embed
from lightrag.utils import EmbeddingFunc
import numpy as np
from lightrag.kg.shared_storage import initialize_pipeline_status

## Constants

In [3]:
WORKING_DIR = "./amazon_product"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

In [ ]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        os.getenv("LLM_MODEL"),
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await openai_embed(
        texts,
        model=os.getenv("EMBEDDING_MODEL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
    )


async def get_embedding_dim():
    test_text = ["This is a test sentence."]
    embedding = await embedding_func(test_text)
    embedding_dim = embedding.shape[1]
    return embedding_dim


# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


# asyncio.run(test_funcs())
async def initialize_rag():
    embedding_dimension = await get_embedding_dim()
    print(f"Detected embedding dimension: {embedding_dimension}")

    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=llm_model_func,
        embedding_func=EmbeddingFunc(
            embedding_dim=embedding_dimension,
            max_token_size=8192,
            func=embedding_func,
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()

    return rag

### System Prompt

In [ ]:
custom_prompt = """
You are an expert assistant in web development and state management, specializing in mapping Redux-like state objects to corresponding DOM elements. 
Provide detailed and structured answers with practical examples that explain how specific state changes should trigger UI updates and endpoint modifications. 

---Conversation History--- 
{history} 

---Knowledge Base--- 
{context_data} 

---Response Rules---

Target format and length: {response_type}
"""

In [ ]:
# Assuming initialize_rag, rag.aquery, and other relevant functions are defined earlier

async def initialize_rag():
    # Initialize your RAG instance (fill in this function with your actual code)
    pass

# Step 1: Initialize RAG
rag = await initialize_rag()

In [ ]:
# Step 2: Read and insert the content from the HTML file
with open("dataset/amazon_product.html", "r", encoding="utf-8") as f:
    await rag.ainsert(f.read())

In [ ]:
# Step 3: Perform a local search
local_search_result = await rag.aquery(
    "Retrieve all DOM elements and UI components associated with the state object key: selectedProduct. Identify which UI elements (e.g., icons, containers and endpoints) should be updated when this state key changes. Return a JSON object mapping the state object key: selectedProduct to an array of objects each detailing the line within the DOM, element's tag, relevant attributes, and inner text if applicable.",
    param=QueryParam(mode="local"),
    system_prompt=custom_prompt  # Pass the custom prompt
)
print(local_search_result)

In [ ]:
# Step 4: Perform a global search
global_search_result = await rag.aquery(
    "Retrieve all DOM elements and UI components associated with the state object key: selectedProduct. Identify which UI elements (e.g., icons, containers) and endpoints should be updated when this state key changes, detailing the specific modifications required.",
    param=QueryParam(mode="global"),
    system_prompt=custom_prompt  # Pass the custom prompt
)
print(global_search_result)

In [ ]:
# Step 5: Perform a hybrid search
hybrid_search_result = await rag.aquery(
    "Retrieve all DOM elements and UI components associated with the state object key: selectedProduct. Identify which UI elements (e.g., icons, containers and endpoints) should be updated when this state key changes. Return a JSON object mapping the state object key: selectedProduct to an array of objects each detailing the line within the DOM, element's tag, relevant attributes, and inner text if applicable.",
    param=QueryParam(mode="hybrid"),
    system_prompt=custom_prompt  # Pass the custom prompt
)
print(hybrid_search_result)